<h2 align="center">DLOps - Tracking Experiments</h2>

Data Scientist.: PhD.Eddy Giusepe Chirinos Isidro

No script anterior aprendimos a executar nossos experimentos apartir de `arquivos de configuração`. Isso nos ajuda a manter um registro de todos nossos experimentos. Não entanto, isto não nos permite saber se um experimento a sido exitoso ou não. Para isso precisamos trackear as diferentes métricas que nos permite evaliar o rendimento de nossos experimentos, sabendo assim qual deles nos dá os resultados que procuramos.

# Tensorboard

Por default `Pytorch Lightning` usa uma ferramenta chamada `Tensorboard` para trackear os resultados dos experimentos. Esta ferramenta nos permite visualizar os resultados dos experimentos numa forma simples. Ademais, graças a sua integração com `VSCode`, podemos visualizar os resultados dos experimentos em uma aba de nosso editor de texto sem necessidade de trocar de ferramenta. Para isso, simplesmente temos que ativar a variável `logger` em nosso treinamento.

No arquivo de configuração `002.yml` simplesmente adicionar o seguinte conteúdo:

```
trainer:
  logger: True
```

Agora podemos executar o treinamento com o comando:

```
python main.py experiments/002.yml
```

Observarás que se criará uma pasta chamada `lightning_logs`, e em `VSCode` sairá um popup para abrir `Tensorboard`. Senão, sempre poderás abrir a "paleta de comandos" (`CTRL + SHIFT + P`) e escrever `tensorboard`. Verifica sempre de selecionar a pasta `lightning_logs`.


Teus experimentos se irão salvando de maneira automática e poderás compará-los entre sim em qualquer momento.

# CSV logger

`Pytorch Lightning` nos oferece a opção de definir outros sistemas de `loggeado`. Um muito útil consiste em salvar os resultados dos experimentos em um arquivo `CSV` local, para logo carregar e visualizar. Para isso podemos modificar nosso script de Treinamento: 

In [ ]:
from src import *
from pathlib import Path
import pytorch_lightning as pl
import yaml
import sys

config = {
    'datamodule': {
        'path': Path('dataset'),
        'batch_size': 25
    },
    'trainer': {
        'max_epochs': 10,
        'enable_checkpointing': False,
        'overfit_batches': 0
    },
    'logger': None,
}


def train(config):
    dm = MNISTDataModule(**config['datamodule'])
    module = MNISTModule(config)
    # configure logger
    if config['logger'] is not None:
        config['trainer']['logger'] = getattr(pl.loggers, config['logger'])(
            **config['logger_params'])
    trainer = pl.Trainer(**config['trainer'])
    trainer.fit(module, dm)
    trainer.save_checkpoint('final.ckpt')


if __name__ == '__main__':
    if len(sys.argv) > 1:
        config_file = sys.argv[1]
        if config_file:
            with open(config_file, 'r') as stream:
                loaded_config = yaml.safe_load(stream)
            deep_update(config, loaded_config)
    print(config)
    train(config)

E no experimento `003.yml` adicionamos o seguinte conteúdo:

```
logger: CSVLogger
logger_params:
  save_dir: logs
  name: "003"
```  

Ao executar o experimento se criará uma pasta `logs` na qual teremos outra pasta com o nome `003` na qual se irão salvando os logs de cada execução do experimento. Obviamente, você pode trocar isto com os parâmetros que desejes. 

In [ ]:
import pandas as pd 

logs = pd.read_csv('logs/003/version_0/metrics.csv')

logs

![](sen_1.png)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(6, 3))
ax = plt.subplot(1,2,1)
logs['val_loss'].dropna().plot(ax=ax)
logs['loss'].dropna().plot(ax=ax)
ax.legend(['val_loss', 'loss'])
ax.grid(True)
ax = plt.subplot(1,2,2)
logs['val_acc'].dropna().plot(ax=ax)
logs['acc'].dropna().plot(ax=ax)
ax.legend(['val_acc', 'acc'])
ax.grid(True)
plt.tight_layout()
plt.show()

![](sen_2.png)

Isto é muito útil quando se trabalha de maneira isolada localmente, não entanto se trabalhas de maneira colaborativa podes ter os logs na nuvem e accesíveis através da internet é muito melhor.

# Weights and Biases

Minha opção mais bacana para `trackear` experimentos é 